<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# Pipelines

En este notebook, vamos a dar los primeros pasos para ver cómo podemos utilizar transformers para distintas tareas de PLN. 

HuggingFace proporciona muchos modelos pre-entrenados que ya están disponibles para ser utilizados directamente. 

Un **pipeline** es una aplicación que encapsula un modelo pre-entrenado y que permite utilizar dicho modelo de forma sencilla. 
La librería **transformer** proporciona una clase **Pipeline** que permite cargar un modelo pre-entrenado para una tareas de PLN específica como: clasificación de textos, análisis de sentimiento, reconocimiento de entidades, etc. 



Nota: En este notebook, no es necesario utilizar GPU o TPU. 

El primer paso será instalar la librería de transformers:



In [ ]:
!pip install transformers


## Clasificación de textos

El pipeline **text-classification** por defecto carga el modelo pre-entrenado 'distilbert-base-uncased-finetuned-sst-2-english', que fue entrenado sobre el corpus SST (análisis de sentimiento con dos clases)

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification")
classifier("The movie was very boring.")



De hecho, ese mismo modelo se carga para el pipeline **sentiment-analysis**:

In [ ]:
classifier = pipeline("sentiment-analysis")
print(classifier("The sandwich is deliciuous."))
print(classifier("I've been waiting for a deep learning course my whole life."))
print(classifier("This is the worst movie ever made."))

También se puede aplicar directamente sobre una lista de oraciones:

In [ ]:
classifier(
    ["I've been waiting for a deep learning course my whole life.", 
     "The food was delicious!!",
     "This is the worst service we've ever had.",
     "Today is Monday",
     "Today is Friday",
     "This restaurant is awesome.", 
     "The movie was very boring."]
)

También puedes cargar modelos específicos. Por ejemplo, como el modelo **cardiffnlp/twitter-xlm-roberta-base-sentiment**, un modelo RoBERTa pre-entrenado con tweets para la tarea de análisis sentimiento. 
Este modelo, y muchos otros, requieren la instalación **sentencepiece** (un tokenizador especial para redes neuronales)

In [ ]:
!pip install sentencepiece

In [ ]:
classifier = pipeline(model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
print(classifier("The hotel was excellent."))
print(classifier("The movie was produced in Spain."))



### Zero-shot-classification 
El tarea de análisis de sentimiento es un problema de clasificación de texto, donde el objetivo es detectar la polaridad (positiva o negativa) de un texto. Sin embargo, existen otros muchos problemas de clasificación de texto como detección de spam, detección de noticias falsas, detección de mensajes de odio, detección de mensajes sexistas, etc. En cada problema, el conjunto de clases (etiquetas) puede ser diferente.

El pipeline **zero-shot-classification** nos va a permitir clasificar cualquier tipo de texto, sin necesidad de entrenar el modelo con un conjunto de datos específico para la tarea. Únicamente es necesario especificar el conjunto de clases. El pipeline va a producir una probabilidad para cada una de las clases. 

In [ ]:
zero_pipeline = pipeline("zero-shot-classification")


Por ejemplo, podemos usar el pipelines para clasificar noticias:

In [ ]:
zero_pipeline(
    "Elon Musk agreed in late April to buy Twitter for $54.20 per share",
    candidate_labels=["education", "politics", "business", "health"],
)

In [ ]:
zero_pipeline(
    "The U.S. government has ordered millions of doses of a vaccine that protects against monkeypox.",
    candidate_labels=["education", "politics", "business", "health"],
)

También para detectar mensajes sexistas o racistas:

In [ ]:
zero_pipeline(
    "All Muslims are terrorists.",
    candidate_labels=["racism", "sexism", "neutral"],
)

In [ ]:
zero_pipeline(
    "Women drive badly unlike men.",
    candidate_labels=["racism", "sexism", "neutral"],
)

La ideología de un tweet:

In [ ]:
zero_pipeline(
    "El aborto es un asesinato.",
    candidate_labels=["derecha", "izquierda", "neutro"],
)

Identificar noticias falses

In [ ]:
zero_pipeline(
    "The Earth is flat",
    candidate_labels=["fake", "non-fake"],
)

El modelo no siempre funciona correctamente!!!



In [ ]:
zero_pipeline(
    "Women are one half of society.",
    candidate_labels=["racism", "sexism", "neutral"],
)

In [ ]:
zero_pipeline(
    "Donald Trump is a left-wing politician",
    candidate_labels=["fake", "non-fake"],
)

Ejemplo de noticia false (https://edition.cnn.com/2022/03/12/politics/fact-check-dicaprio-donation-10-million-ukraine/index.html)

In [ ]:
zero_pipeline(
    "Leonardo DiCaprio makes multiple donations to Ukraine",
    candidate_labels=["fake", "non-fake"],
)

In [ ]:
zero_pipeline(
    "All babies are precious! Life starts at conception! Pro-Life!.",
    candidate_labels=["right", "left", "neutral"],
)

## Generación de textos

Esta tarea consiste en producir texto nuevo. Son modelos útiles para tareas donde es necesario completar un texto o para tares como la generación de resúmenes, paráfrisis, etc. 

ChatGPT es un ejemplo de este tipo de modelo. Por ejemplo, es capaz de a partir de la descripción de un código, generar el código. 

Dentro de estos modelos, podemos distinguir dos tipos:
- los que a partir de un texto, tratan de predecir la siguiente palabra. Los modelos GPT son de este tipo. 
- los que traducen un texto a otro (por ejemplo, generan un resumen). T5 o BART son algunos de los modelos más populares. 

HugginFace ya proporciona un pipeline que nos permite genera texto nuevo sin necesidad de entrenarlo sobre ningún corpus. En concreto, el pipeline recibe un texto y lo completa. 



In [ ]:
generator = pipeline("text-generation")


Text generation involves randomness, so the model will produce different outputs every time: 

In [ ]:
generator("Protest erupt in Greece")


You can set the number of outputs and the maximum lenght of each sentence. 


In [ ]:
generator(
    "once upon a time there was a little girl",
    max_length=20,
    num_return_sequences=4,
)

Podemos especificar el modelo que queremos utilizar para la tarea de generación (https://huggingface.co/models?pipeline_tag=text-generation)
En este caso usaremos, el modelo distilgpt2:

In [ ]:
generator = pipeline("text-generation", model="distilgpt2")

In [ ]:
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

VAmos a cargar un modelo para español:

In [ ]:
generator = pipeline("text-generation", model="DeepESP/gpt2-spanish")

In [ ]:
generator(
    "El gobierno cree que la huelga será ",
    max_length=30,
    num_return_sequences=2,
)

In [ ]:
generator(
    "Erase una pequeña niña",
    max_length=30,
    num_return_sequences=2,
)

### Mask-filling

Otra tarea relacionada con la generación de textos, es la tarea capaz de inferir los huecos en una oración. El pipeline es **fill-mask**.

In [ ]:
unmasker = pipeline("fill-mask")


Debemos utilizar el token especial  $<mask>$ para indicar qué huecos deben ser completado. El parámetro **top_k** indica cuántas posibilidades queremos recuperar:

In [ ]:
unmasker("In this course, I will teach you how to develop <mask> models.", top_k=5)

In [ ]:
unmasker("There was a <mask> girl.", top_k=2)

Vamos a ver cómo funciona cuando hay varios huecos:

In [ ]:
unmasker("In this <mask>, I will teach you how to develop <mask> models.", top_k=5)

## Reconocimiento de entidades
En inglés, Named Entity Recognition (NER). El objetivo es identifica menciones de personas, lugares y organizaciones. 

In [ ]:
ner = pipeline("ner", grouped_entities=True)


In [ ]:
ner("My name is Isabel Segura. I work at Carlos III University of Madrid.")


### Ejercicio:
¿Qué efecto tiene grouped = False?

In [ ]:
ner("My name is Isabel. I work at Carlos III University of Madrid", grouped_entities = False )


Vamos a cargar un modelo pre-entrenado para reconocer entidades en textos en español

In [ ]:
ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner',  {"use_fast": False}
))


In [ ]:
sentence="El PP sería primera fuerza en Sevilla y Granada"
ner(sentence, grouped_entities = True)

In [ ]:
sentence="Moreno Bonilla necesitará a VOX para gobernar en Andalucía"
ner(sentence, grouped_entities = False)

## Question answering
Este pipeline permite responder preguntas utilizando la información de un contexto: 


In [ ]:
question_answerer = pipeline("question-answering")


In [ ]:
question_answerer(
    question="Where do I work?",
    context="My name is Isabel and I work at Carlos III University of Leganés",
)

In [ ]:
question_answerer(
    question="¿Cuál es la tasa de paro en España?",
    context="El paro de la eurozona repite el mínimo histórico del 6,8%, mientras España sigue en el 13,3%",
)

## Generación de resúmenes
Genera un resúmen preciso y conciso para un texto:

In [ ]:
summarizer = pipeline("summarization")

La generación del resumen puede tardar unos minutos:

In [ ]:
text = """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""

summarizer(text)

## Traducción automática

Dado un texto en un idioma, el modelo genera su traducción a otro idioma. Es necesario instalar la librería **sentencepiece**


In [ ]:
translator = pipeline("translation_en_to_fr")
translation = translator(text)

print(translation)

In [ ]:
text = "Hello my friends! How are you doing today?"

translator = pipeline("translation_en_to_de")
translation = translator(text)

print(translation)

## Feature extraction
Este pipeline te permite extraer los vectores que el modelo pre-entrenado asignaría a la oración de entrada. Por defecto, utiliza el modelo **distilbert-base-cased**, aunque podríamos probar con otro modelo. 

In [ ]:
#text = "Hello my friends"
#text = "Hello my friends! How are you doing today?"

extractor = pipeline("feature-extraction")
vector=extractor(text)


Vemos la dimensión de la palabra (siempre es 768 en un modelo de Distilbert)



In [ ]:
print(len(vector[0][0])) # should be 768

assert len(vector[0][0]) == 768 # los vectores de las palabras tienen dimensión 768

Para aumentar la información sobre pipelines, por favor, consulta la siguiente página https://huggingface.co/docs/transformers/main_classes/pipelines


## Ejercicio 1: 

Prueba con el modelo **finiteautomata/beto-sentiment-analysis** para la tarea de análisis de sentimiento en español.

In [ ]:
...

## Ejercicio 2: 

Crea un pipeline que te permita clasificar mensajes tóxicos en español en Twitter (busca el modelo en HuggingFace):

In [ ]:
...

## Ejercicio 3: 
Crea un pipeline que te permita clasificar mensajes de bulling en Twitter:

In [ ]:
...


### PRACTICA 1:

Utilizar el pipeline de zero-classification para clasificar los mensajes del conjunto test de EXIST, tanto para la tarea de clasificaicón binaria, como para la tarea de muliclasificación.

Cálcula las métricas y compara con los enfoques anteriores (SVM, CNN con y sin word embeddings, BiLSTM con y sin embeddings, etc). 

¿Cuál es el mejor modelo hasta el momento?

